In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import Phylo
import mysql.connector as SQLC
import mysql.connector
from datetime import date

### Start povoation:

In [59]:
#Extrair ids:

import time
from datetime import datetime


data_e_hora_atuais = datetime.now()

query= input('escolha o que quer pesquisar: ')
def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

import requests, sys, json
content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
#print(content)

import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
    
import re
existe = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(existe)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20): '))]
numero_genes= len(n_genes)
print(n_genes)




escolha o que quer pesquisar: gli
escolha o nº de genes que quer obter (máximo 20): 5
['34927', '30097', '2735', '778919', '108020932']


In [60]:
#Povoação "History"

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor_2 = DataBase.cursor()
try:
    sql= "INSERT INTO History (search, Genes_number_input, Day)   VALUES (%s, %s, %s)"
    val=(query, numero_genes, data_e_hora_atuais)
    Cursor_2.execute(sql,val)
    
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
    search_id =[]
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor_2 = DataBase.cursor()
try:
    sql= "Select ID_search FROM History"
    Cursor_2.execute(sql)
    for row in Cursor_2:
        #print(row)
        search_id.append(str(row))       
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()
    
div= ', '.join(search_id)
h= div.replace("(",'')
hh= h.replace(",)",'')
SEARCH_ID= hh.split(', ')
Hist= SEARCH_ID[-1]

None


In [61]:
#Extrair ids NCBI:

Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
    #print(info.id)
print (Ids)

['Z11228.1', 'X65481.1', 'R24031.1', 'AAPY01624519.1']


In [62]:
import time
def url_get_id(i):
    url_list= [ ]
    for id in i:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
        url_list.append(url)
    return url_list
link_genebank= url_get_id(Ids)
print(link_genebank)

['https://www.ncbi.nlm.nih.gov/nuccore/Z11228.1', 'https://www.ncbi.nlm.nih.gov/nuccore/X65481.1', 'https://www.ncbi.nlm.nih.gov/nuccore/R24031.1', 'https://www.ncbi.nlm.nih.gov/nuccore/AAPY01624519.1']


In [63]:
#Extrair description NCBI:

database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
description=[]
for info in records:
    #print(info.id)
    #print(info.description)
    description.append(info.description)
    print (description)

['H.sapiens gamma-delta T-cell Vgamma9+ mRNA']
['H.sapiens gamma-delta T-cell Vgamma9+ mRNA', 'F.sambucinum (NRRL 13700, 13504) internal transcribed spacer including 5.8S rDNA gene']
['H.sapiens gamma-delta T-cell Vgamma9+ mRNA', 'F.sambucinum (NRRL 13700, 13504) internal transcribed spacer including 5.8S rDNA gene', "yh29h04.r1 Soares placenta Nb2HP Homo sapiens cDNA clone IMAGE:131191 5', mRNA sequence"]
['H.sapiens gamma-delta T-cell Vgamma9+ mRNA', 'F.sambucinum (NRRL 13700, 13504) internal transcribed spacer including 5.8S rDNA gene', "yh29h04.r1 Soares placenta Nb2HP Homo sapiens cDNA clone IMAGE:131191 5', mRNA sequence", 'Tupaia belangeri voucher Broad:Sequencing.Sample:10448, Broad Institute, Cambridge, MA cont1.624518, whole genome shotgun sequence']


In [64]:
#organismos

Organismos=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Organismos.append(info.annotations['organism'])
print(Organismos)

['Homo sapiens', 'Fusarium sambucinum', 'Homo sapiens', 'Tupaia belangeri']


In [50]:
#Biopython
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    print(info.id)
    print(info.seq[0:10],'...',info.seq[-10:])
    print()
    seq.append(info.seq[0:10])

AL329769.1
GGGACAGATC ... CCCCANTCTT

X61083.1
AACTCCTTCC ... GAGGCCGCAA

U02356.1
CCCCGAGGCC ... GGTTGGAGAC

AASW01042081.1
ATTTTGTCAT ... GAATCCACCA

Y00554.1
GTCGACAACG ... CTTGTCGCTT

EL279606.1
GAAAGGGAAT ... ACAGAAGACA

EC941813.1
TTGATTTCAT ... GGGTCCCTTG

X61884.1
GGCGATGTGT ... ACATGTGATG

AAADZ0085841.1
GGGGTGTGGC ... GTGGATTTGC



In [65]:
Adenina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('A')
    perc= int(count/len(c)*100)
    Adenina.append(perc)
print (Adenina)

[31, 27, 23, 30]


In [66]:
Citosina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('C')
    perc= int(count/len(c)*100)
    Citosina.append(perc)
print (Citosina)

[26, 25, 19, 18]


In [67]:
Guanina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('G')
    perc= int(count/len(c)*100)
    Guanina.append(perc)
print (Guanina)

[15, 23, 24, 18]


In [68]:
Timina=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    c=info.seq
    count= c.count('T')
    perc= int(count/len(c)*100)
    Timina.append(perc)
print (Timina)

[26, 24, 30, 32]


In [69]:
data=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    data.append(info.annotations['date'])
print(data)

['09-FEB-1999', '02-SEP-1992', '20-APR-1995', '06-JUN-2006']


In [70]:
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
tam=[]
for info in records:
    tam.append(len(info.seq))

print(tam)

[210, 556, 411, 941]


In [71]:
#Permitir a povoação em uma coluna de cada vez:

import mysql.connector as SQLC
import mysql.connector

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try: 
    Cursor.execute("SET GLOBAL sql_mode = ''")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [72]:
#Povoação "Gene"

import mysql.connector as SQLC
import mysql.connector

#tab = ("INSERT INTO Gene (ID_genebank) VALUES ('{}')".format(value)) # %s overwrites the same value

des = (description)
DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(Ids):
        #print(value)
        #print(des[index])
        sql= "INSERT INTO Gene (ID_genebank, Description, Organism, length, Adenina, Citosina, Guanina, Timina, Link, ID_search) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val=(value, des[index], Organismos[index], tam[index], Adenina[index], Citosina[index], Guanina[index], Timina[index], link_genebank[index], Hist)
#         print(type(des[index]))
        Cursor.execute(sql,val)
#         for values in des:
#             Cursor.execute(f"INSERT INTO Gene ( Description)  VALUES ('{values}')")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [ ]:
#Vizualizar tabela History:Erro na escrita na base de dados: 1062 (23000): Duplicate entry '0' for key 'PRIMARY'


import pandas as pd 

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)

####### end of connection ####
tabela = pd.read_sql("Select * FROM Gene", DataBase)
DataBase.close()
print(tabela)